In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
import xgboost as xgb
import sys
import kaggle_xgb
import calc_ndcg
import itertools
import dataEngr
reload(dataEngr)
reload(calc_ndcg)
from sklearn.grid_search import ParameterGrid

Read in just the testing and training data.

In [2]:
xgbInput = dataEngr.clfInput()
xgbInput.get_sessionsFtr()
xgbInput.users_ftrEng()
xgbInput.one_hot()
#xgbInput.binarize_targets()
#xgbInput.split_bySess()

number of columns before one hot encoding 370
number of columns after one hot encoding 469


In [4]:
xgbInput.allDf

,10_message_post,11_message_post,12_message_post,15_message_post,about_us,account,acculynk_bin_check_success,acculynk_load_pin_pad,acculynk_pin_pad_inactive,acculynk_session_obtained,...,first_browser_Opera Mini,first_browser_Opera Mobile,first_browser_Pale Moon,first_browser_Safari,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_Sogou Explorer,first_browser_Yandex.Browser,first_browser_wOSBrowser
00023iyk9l,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
0005ytdols,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
000guo2307,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
000wc9mlv3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,1,0,0,0,0,0,0
0010k6l0om,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0012yo8hu2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
001357912w,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
001nvbxsvp,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
001wyh0pz8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001xf4efvm,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data = pd.read_csv('../data/train_users_2.csv', index_col = 'id')
test_data = pd.read_csv('../data/test_users.csv', index_col = 'id')
sessionsDf = pd.read_csv('../data/sessions.csv')
sessionsDf.dropna(subset=['user_id'], inplace = True)

In [ ]:
for i in sessionsDf.action.unique():
    if len(sessionsDf[sessionsDf.action == i].action_detail.unique())>1:
        print i, sessionsDf[sessionsDf.action == i].action_detail.unique()

In [ ]:
train_actions = sessionsDf[sessionsDf.user_id.isin(train_data.index)].action.unique()
test_actions = sessionsDf[sessionsDf.user_id.isin(test_data.index)].action.unique()
a_ignore = set(train_actions) ^ set(test_actions)

In [ ]:
%run -i sessionsFtr.py

In [ ]:
#get count for actions in test session
for a in a_ignore:
    if testcnts(a, test_sessions.action)>0:
        print a, test_sessions[test_sessions.action== a].action_detail.unique(), \
            test_sessions[test_sessions.action== a].action_type.unique(), testcnts(a, test_sessions.action)

In [ ]:
for i in tr_sessions.action.unique():
    if 'book' in str(i):
        print i

In [ ]:
test_sessions = sessionsDf[sessionsDf.user_id.isin(test_data.index)]
tr_sessions = sessionsDf[sessionsDf.user_id.isin(train_data.index)]
tr_sessions.set_index('user_id', drop= False, inplace=True)
tr_sessions = pd.concat([tr_sessions, train_data.country_destination], join = 'inner', axis = 1)

In [ ]:
post_cnts = tr_sessions.groupby(['user_id']).apply(lambda x: np.sum(x['action_type'] == 'message_post'))

In [ ]:
train_df = pd.concat([train_df, posters_cnts], axis = 1, join = 'inner')
train_df = train_df.rename(columns = {0:'post_cnts'})

In [ ]:
#31358 users posted messages in test set, it's worth distinguishing NDF and non-NDF?
#fifty-fifty split
print len(test_sessions.query('action_type == "message_post"').index.unique())
posters = tr_sessions.query('action_type == "message_post"').index.unique()
train_data.loc[posters, 'country_destination'].value_counts(normalize = True)

In [ ]:
#8067 booking requests in test set, it's worth distinguishing NDF and non-NDF
print len(test_sessions.query('action_type == "booking_request"').index.unique())
train_data.loc[all_bookers, 'country_destination'].value_counts(normalize = True)

In [ ]:
all_bookers = tr_sessions.query('action_type == "booking_request"').index.unique()
users_NDF = tr_sessions.query('action_type == "booking_request" & country_destination == "NDF"').index.unique()
users_USA = tr_sessions.query('action_type == "booking_request" & country_destination == "US"').index.unique()
#tr_sessions[tr_sessions.action_type == 'booking_request'].groupby('user_id').filter(lambda x: len('x.country_destination')>1)

In [ ]:
#set hierarchical index???
x = tr_sessions.groupby('user_id').count().country_destination
#groupby reorders user_id, so put back in the original order
x = x.loc[tr_sessions.user_id.unique()]
pos = [range(i) for i in x.tolist()]
pos = list(itertools.chain.from_iterable(pos))
tr_sessions['pos'] = pos
len(pos) == len(tr_sessions)

In [ ]:
#position normalized by total number of actions
pos_norm = [np.arange(total)/float(total-1) for total in x.tolist()]
pos_norm = list(itertools.chain.from_iterable(pos_norm))
tr_sessions['pos_norm'] = pos_norm
len(pos_norm) == len(tr_sessions)

In [ ]:
tr_sessions[tr_sessions.action == 'add_guests'].groupby('user_id')['country_destination'].unique()

In [ ]:
getcntr('ajax_payout_split_edit', tr_sessions.action, train_data)

In [ ]:
getcntr('ajax_google_translate', tr_sessions, train_data)

In [ ]:
getcntr('travel_plans_current', tr_sessions.action, train_data)

In [ ]:
getcntr('connect', tr_sessions.action, train_data)

In [ ]:
getcntr2('booking_request', 'at_checkpoint', tr_sessions.action_type, tr_sessions.action, train_data)

In [ ]:
def testcnts(q, test_series):
    users = test_series[test_series == q].index.unique()
    return len(users)

def getcntr(q, tr_series, train_data):
    users = tr_sessions[tr_series == q].index.unique()
    return train_data.loc[users].country_destination.value_counts()

def getcntr2(q1, q2, tr_series1, tr_series2, train_data):
    users1 = tr_sessions[tr_series1 == q1].index.unique()
    users2 = tr_sessions[tr_series2 == q2].index.unique()
    users = set(users1) & set(users2)
    return train_data.loc[users].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'travel_plans_current'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'place_worth'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'payout_update'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'guest_billing_receipt'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'booking'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'guest_booked_elsewhere'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'transaction_history'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'receipt'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'countries_option'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'itinerary'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'travel_plans_previous'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'cancel'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'office_location'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action_detail == 'your_trips'].country_destination.value_counts()

In [ ]:
users = tr_sessions[tr_sessions.action == 'host_summary'].user_id

In [ ]:
tr_sessions[tr_sessions.action == 'phone_number_widget'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'payment_instruments'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'agree_terms_uncheck'].country_destination.value_counts(normalize = True)

In [ ]:
test_sessions[test_sessions.action == 'print_confirmation']

In [ ]:
options = ['recommend', 'recommendations', 'references', 'referrer_status']
for i in options:
    print tr_sessions[tr_sessions.action == i].country_destination.value_counts()

In [ ]:
axes = plt.gca()
axes.set_xlim([0,20000])
s.hist(ax=axes, bins= 10000)

In [ ]:
s2 = tr_sessions.groupby('user_id').filter(lambda x: len(x['action'])>3)

In [ ]:
s2.groupby('user_id').apply(lambda x: x.iloc[1:2, :] )

In [ ]:
s2[s2.country_destination == 'NDF'].action.value_counts()

In [ ]:
s = sessions.groupby('user_id')['secs_elapsed'].apply(lambda x:  x.iloc[-1])

In [ ]:
train_actions = sessions[sessions.user_id.isin(train_data.index)].action.unique()
test_actions = sessions[sessions.user_id.isin(test_data.index)].action.unique()
a_ignore = set(train_actions) ^ set(test_actions)

In [ ]:
actions50 = [sessions_test.groupby('action')['user_id'].nunique() >50]
print 'Number of unique actions with more than 50 users:', np.sum(sessions_test.groupby('action')['user_id'].nunique()>50)

In [ ]:
#Let's get the actions with just one action type and one action detail.
print 'Number of unique actions with just one action type:',
np.sum(sessions.groupby('action')['action_detail'].nunique()==1)
#Take the intersection of these actions with a_ignore.

In [ ]:
sessions['action_type'] = sessions['action_type'].fillna('missing')
sessions['action_detail'] = sessions['action_detail'].fillna('missing')

In [ ]:
#actions with more than one action_detail
#sessions['action_type'] = sessions['action_type'].fillna('none')
#sessions['action_detail'] = sessions['action_detail'].fillna('none')

def custom_filter(x):
    conditions = (x['action_detail'].nunique() > 1) or (x['action_type'].nunique() > 1)
    if conditions:
        return True
actions_df = sessions.groupby('action').filter(lambda x: custom_filter(x))
for i in actions_df.action.unique():
    print i, sessions[sessions.action==i].action_type.unique(), sessions[sessions.action==i].action_detail.unique()

In [ ]:
#exclude those actions that are exclusively found in one table or the other
sessions = sessions[~sessions.action.isin(a_ignore)]

In [ ]:
#info = actions_1.groupby(['user_id', 'action']).apply(lambda x: x['secs_elapsed'].sum())
action_cnts = sessions.groupby(['user_id', 'action']).apply(lambda x: x['device_type'].count())

In [ ]:
action_cnts = action_cnts.unstack(level=-1)

In [ ]:
action_cnts.fillna(0, inplace=True)

In [ ]:
users = pd.concat([allDf, action_cnts], axis = 1, join = 'inner').index

In [ ]:
pd.to_pickle(action_cnts, '../data/actions.p')

In [ ]:
pd.DataFrame(,index = sessions.user_id[])

In [ ]:
#find the number of users in test data with an action that is not found in the training sessions data
#We should look to find similar items to substitute them
for i in a_ignore:
    users = sessions_test[sessions_test.action == i].user_id.unique()
    if len(users) > 0:
        print i, len(users)

In [ ]:
tr_sessions[tr_sessions['user_id'] == '1etkf91jg7']

In [ ]:
getcntr('apply_reservation', tr_sessions.action, train_data)

In [ ]:
tr_sessions.query("action == 'apply_coupon_error' & pos_norm > .9")

In [ ]:
def get_counts(x):
    return sessionsDf.action_detail.value_counts()['message_post']
        
sessionsDf.groupby(['user_id']).apply(lambda x: get_counts(x))

In [ ]:
dev_prop = sessions.groupby(['user_id']).apply(lambda x: x['device_type'].value_counts(normalize = True))

In [ ]:
pd.concat([allDf, dev_counts], join = 'outer')

In [ ]:
dev_countsDf = dev_counts.unstack()

In [ ]:
#merge these actions with action_detail
mergedActs = sessions.action + '_' + sessions.action_detail
#ignoring patch
#ignoring 'pending'
actions = ['index', 'show', 'create', 'reviews', 'delete', 'recommendations', 'update']
for a in actions:
    sessions.loc[sessions.action == a, 'action'] = mergedActs[sessions.action == a]

In [ ]:
#iterate through each action and find those that have more than one action detail/ action type
#s = sessions.groupby('action')['action_type'].nunique()
#sessions.groupby('action')['action_detail'].nunique()

In [ ]:
ssh -i "amazon1.pem" root@ec2-54-200-37-178.us-west-2.compute.amazonaws.com


In [ ]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]